In [ ]:
#前置作業準備，只要一次!!
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

df5k = pd.read_json("paras/mapping5K.json")
dfv = pd.read_csv("volumeData/ma_20230110.csv")
dfv["id"] = dfv["id"].astype("string")
# dfv = dfv.drop(columns=['close'])
print(dfv[dfv["id"] == "1305"])

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}

url4 = "https://www.wantgoo.com/investrue/all-alive"
r3 = requests.get(url4, headers = headers).content
soup = BeautifulSoup(r3, "html.parser")
rr3 = soup.prettify()
dfn = pd.read_json(rr3)
dfn = dfn[(dfn["id"]>="1101") & (dfn["id"]<="9999") & (dfn["id"].str.len() == 4)]
dfn = dfn.drop(columns=['type','country','url', 'industries'])
dfn["id"] = dfn["id"].astype("string")

### 計算量比參數 ###
import time
def getRate5K():
    localtime = time.localtime()
    nowTime = time.strftime("%H:%M", localtime)
    Rate5k = df5k[(df5k["e"] >= nowTime) & (df5k["b"] <= nowTime)]["w"].values[0]  #五分K量比預估放大參數 
    print(f"目前時間：{nowTime},預估量放大係數:{Rate5k}")
    return Rate5k

print(getRate5K())

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import _beowFmt as fm 
from IPython.display import display

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 500)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.width', 180)                       # 设置打印宽度(**重要**)
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}

r = requests.get("https://www.wantgoo.com/investrue/all-quote-info", headers = headers).content
soup = BeautifulSoup(r, "html.parser")
rr = soup.prettify()
df = pd.read_json(rr)
df = df.drop(columns=['tradeDate', 'time','millionAmount'])
df["id"] = df["id"].astype("string")
df = pd.merge(df, dfn, left_on="id", right_on="id")  ## 結合股票名稱
df["float"] = df["close"].astype("float")
df["amp"] = ((df["close"] - df["open"])/df["open"] * 100).round(2)
df["jump"] = df["open"] - df["previousClose"]
df["jumpRate"] = (df["jump"] / df["open"] * 100).round(2)
df = df[(df["id"]>="1101") & (df["id"]<="9999") & (df["id"].str.len() == 4) & (df["close"] > df["previousClose"])].sort_values("jumpRate", ascending=False)
# df.info()
# print(df.head(50))

r2 = requests.get("https://www.wantgoo.com/stock/all-turnover-rates", headers = headers).content
soup = BeautifulSoup(r2, "html.parser")
rr2 = soup.prettify()
dfa = pd.read_json(rr2)
# print(dfa)

dfa["investrueId"] = dfa["investrueId"].astype("string")
dfb = pd.merge(df, dfa, left_on="id", right_on="investrueId")
dfb = dfb.drop(columns=['investrueId'])
dfb = dfb[(dfb["value"] > 1)]
dfb["周轉率"] = dfb["value"].round(2)


dfc = pd.merge(dfb, dfv, left_on="id", right_on="id")
dfc["預估量"] = (dfc["volume"] * getRate5K()).astype(int)        ##預估量
dfc["量比"] = (dfc["預估量"] / dfc["yVolume"]).round(2)
dfc["週量比"] = (dfc["預估量"] / dfc["ma5"]).round(2)
dfc["月量比"] = (dfc["預估量"] / dfc["ma20"]).round(2)
dfc["季量比"] = (dfc["預估量"] / dfc["ma60"]).round(2)
dfc["量比周轉"] = (dfc["量比"] + dfc["value"]) / 2
dfc2 = dfc[dfc["周轉率"] > 1]
df3 = dfc2.loc[:, ["id","market","name","open","close","amp","jump","jumpRate","yVolume","預估量","量比","週量比","月量比","季量比","周轉率","量比周轉"]].sort_values("量比周轉", ascending=False)
display(df3)

ss = ''
for d in dfc2["id"].tolist():
  ss += f"{d}.TW,"
fm.write_LogFile(f"xq_import/量價型態20230110.csv", ss)



In [ ]:
## 自己擷取數量，只用來與昨天比較...
import _beowFmt as fm 
dd = dfb[["id","close","volume"]].sort_values("id")
ss = 'id,close,yVolume\n'
for d in dd.values.tolist():
  ss += f"{d[0]},{d[1]},{d[2]}\n"
fm.write_LogFile(f"volumeData/20230110.csv", ss)

# todo ##
# https://www.wantgoo.com/investrue/all-previous-weekly-candlestick?v=1672966860000  上一周收盤價


In [ ]:
## 找出今天數量，各種均量
import finlab
finlab.login(open("config.txt", "r").read())

from finlab import data
import pandas as pd
import _beowFmt as fm 

def combineStr(r):
    stockId, ma1, ma5, ma20, ma60 = r["stockId"], r["ma1"], r["ma5"], r["ma20"], r["ma60"]
    return f"{stockId},{ma1},{ma5},{ma20},{ma60}"

##
volumns  = data.get("price:成交股數").fillna(0).astype(float)[240: ]  
v = (volumns/1000).round(0).astype(int)
vMa5 = v.rolling(5, min_periods=1).mean().round(0).astype(int).tail(1)
vMa20 = v.rolling(20, min_periods=1).mean().round(0).astype(int).tail(1)
vMa60 = v.rolling(60, min_periods=1).mean().round(0).astype(int).tail(1)

vdf = v.iloc[-1:].transpose()
vdf["stock"] = vdf.index
vdf.columns = ["ma1", "stockId"]
vdf["ma5"] = vMa5.iloc[-1:].transpose()
vdf["ma20"] = vMa20.iloc[-1:].transpose()
vdf["ma60"] = vMa60.iloc[-1:].transpose()
vdf["comp"] = vdf.apply(combineStr, axis = 1)
vdf2 = vdf[ (vdf["stockId"].str.len() == 4)].dropna(axis = 0, how ='any')

print(vdf)
s1 = 'id,yVolume,ma5,ma20,ma60\n'
for c in vdf2["comp"].tolist():
    s1 += f"{c}\n"
# print(s1)
fm.write_LogFile(f"volumeData/ma_20230110.csv", s1)

In [ ]:
import time
localtime = time.localtime()
nowTime = time.strftime("%H:%M", localtime)
print(nowTime)
# multiple_level_data = pd.json_normalize(data, record_path =['Results'], meta =['original_number_of_clusters','Scaler','family_min_samples_percentage'], meta_prefix='config_params_', record_prefix='dbscan_')

In [ ]:
import _beowFmt as fm 

df2 = df[(df["id"]>="1101") & (df["id"]<="9999") & (df["volume"] >= 1000)].sort_values(by=['id'], ascending=True)
dd = df2[{"id","volume"}]
ss = ''
for d in dd.values.tolist():
  ss += f"{d[1]}, {d[0]}\n"

fm.write_LogFile(f"keepData/stockVolume.csv", ss)

In [ ]:
while True:
    localtime = time.localtime()
    result = time.strftime("%Y-%m-%d %I:%M:%S %p", localtime)
    print(result)
    time.sleep(1)

In [ ]:
from pathlib import Path  

filepath = Path('https://www.wantgoo.com/investrue/all-quote-info')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)  

In [ ]:
# importing the requests library
import requests

# api-endpoint
URL = "http://maps.googleapis.com/maps/api/geocode/json"

# location given here
location = "delhi technological university"

# defining a params dict for the parameters to be sent to the API
PARAMS = {'address':location}

# sending get request and saving the response as response object
r = requests.get(url = URL, params = PARAMS)

# extracting data in json format
data = r.json()


# extracting latitude, longitude and formatted address
# of the first matching location
latitude = data['results'][0]['geometry']['location']['lat']
longitude = data['results'][0]['geometry']['location']['lng']
formatted_address = data['results'][0]['formatted_address']

# printing the output
print("Latitude:%s\nLongitude:%s\nFormatted Address:%s"
	%(latitude, longitude,formatted_address))
